In [36]:
#import libraries

import numpy as np
import pandas as pd

#data stuff
#import data with pandas
df = pd.read_csv('crisis_dataset.csv')

#data cleaning

# drop notes column, drop complexity, drop A1 Nukes and A2 Nukes, drop name
df.drop(['NOTES','COMPLEXITY','A1 NUKES','A2 NUKES','NAME'], axis=1, inplace=True)

#dropna??
#df.dropna(inplace=True)

#iterate through crisis, history, row for each crisis
first_column = df['CRISNO']
second_column = df['HISTORY']

unique_crisis_no = first_column.unique()
final_selected_rows = []

# iterate over crisis nos
for first_value in unique_crisis_no:
    # this filters data for unique crisis no
    filtered_df_first = df[df[first_column] == first_value]
    
    # this initializes the unique history for each crisis
    unique_history = filtered_df_first[second_column].unique()
    chosen_history = np.random.choice(unique_history)

    final_filtered_df = filtered_df_first[filtered_df_first[second_column] == chosen_history]

    # randomly select from filtered data
    final_selected_row = final_filtered_df.sample(n=1)

    # append to final list
    final_selected_rows.append(final_selected_row)

# combine to df
final_combined_df = pd.concat(final_selected_rows)

print(final_combined_df)

#model stuff






KeyError: "None of [Index([  nan,   nan, 445.0, 446.0, 447.0, 448.0, 448.0, 450.0, 450.0, 450.0,\n       ...\n       170.0, 170.0, 170.0, 170.0, 170.0, 171.0, 172.0, 173.0, 175.0, 175.0],\n      dtype='float64', length=996)] are in the [columns]"